In [ ]:
#default_exp config
%load_ext autoreload
from nbdev.showdoc import show_doc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Configuration data and basic functions
> Basic functions and configuration stuff

## time conversion

In [ ]:
#export
from astropy.time import Time
from astropy.coordinates import SkyCoord
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple
import os, sys
import numpy as np

In [ ]:
#export

day = 24*3600.
first_data=54683

def MJD(met):
    "convert MET to MJD"
    #mission_start = Time('2001-01-01T00:00:00', scale='utc').mjd
    # From a FT2 file header
    # MJDREFI =               51910. / Integer part of MJD corresponding to SC clock S
    # MJDREFF =  0.00074287037037037 / Fractional part of MJD corresponding to SC cloc
    mission_start = 51910.00074287037
    return (mission_start + met/day  )

def UTC(mjd):
    " convert MJD value to ISO date string"
    t=Time(mjd, format='mjd')
    t.format='iso'; t.out_subfmt='date_hm'
    return t.value

In [ ]:
assert UTC(MJD(0))=='2001-01-01 00:01'

## Configuration data classes


In [ ]:
#export
@dataclass
class Config:
    """Default light curve configuration parameters"""
    verbose : int = 3
    use_cache : bool = True

    # photon selection
    mjd_range : Tuple=None
    radius: float = 5
    cos_theta_max:float=0.4
    z_max : float=100

    # binning
    energy_edges = np.logspace(2,6,17)
    time_interval: int = 1

    # healpix data representation used by data
    nside : int=1024
    nest: bool=True

    # exposure calculation
    bins_per_decade: int=4
    base_spectrum: str='lambda E: (E/1000)**-2.1'
    energy_range: Tuple = (100.,1e6)
        
    # analysis
    likelihood_rep: str='poisson'
        
    def __str__(self):
        s = 'Configuration parameters \n'
        for name, value in self.__dict__.items():
            s += f'  {name:15s} : {value}\n'
        return s
    
    def __repr__(self): return str(self)

In [ ]:
#
Config()

Configuration parameters 
  verbose         : 3
  use_cache       : True
  mjd_range       : None
  radius          : 5
  cos_theta_max   : 0.4
  z_max           : 100
  time_interval   : 1
  nside           : 1024
  nest            : True
  bins_per_decade : 4
  base_spectrum   : lambda E: (E/1000)**-2.1
  energy_range    : (100.0, 1000000.0)
  likelihood_rep  : poisson

In [ ]:
#export
import pickle
class _Cache(dict):
    
    def __init__(self, path, clear=False):
        self.path = Path(path)
        self.index_file = self.path/'index.pkl'

        if self.path.exists():  
            if clear: 
                print('Clearing cache!')
                self.clear()
            else:
                self._load_index()
        else:
            self.path.mkdir()        
        self.index = len(list(self.path.iterdir()))

    def _dump_index(self):
        with open(self.index_file, 'wb') as file:
            pickle.dump(self, file)
        
    def _load_index(self):
        if not self.index_file.exists(): 
            self._dump_index()
            return
        with open(self.index_file, 'rb') as file:
            self.update(pickle.load(file))
        
    def add(self, key, object, exist_ok=False):
        if key  in self:
            if not exist_ok:
                print(f'Warning: cached object for {key} exists', file=sys.stderr)
            filename = self[key]
        else:
            filename = self.path/f'cache_file_{hex(key.__hash__())[2:]}.pkl'
            self[key] = filename
            self._dump_index() 
 
        with open(filename, 'wb') as file:
            pickle.dump(object, file )

        
    def get(self, key):
        if key not in self:
            return None
        filename = self[key]
        with open(filename, 'rb') as file:
            ret = pickle.load(file)
        return ret
    
    def clear(self):
        
        for f in self.path.iterdir():
            if f.is_file: 
                f.unlink()
        super().clear()

        self._dump_index()
        
    def remove(self, key):
        """remove entry and associated file"""
        if key not in self:
            print(f'Cache: key {key} not found', file=sys.stderr)
            return
        filename = self[key]
        filename.unlink()
        super().pop(key)
        self._dump_index()
        
            
                
    def __call__(self, key, 
                 func:'user function',
                 overwrite:'set to overwrite if exists'=False, 
                 *pars, **kwargs
                ):
        """
        One-line usagge
        
        cache = _Cache(path)
        result = cache(key, function, *pars, **kwargs)
        """
        ret = self.get(key)
        if ret is None or overwrite: 
            ret = func(*pars, **kwargs)
            self.add(key, ret)
        return ret

In [ ]:
#hide
c = _Cache('/tmp/test_cache', clear=True)

c.add('one', 'one');
c.add('two', 'two')
c.add('two', 'two')

ret = c.get('three') 
if ret is None:
    ret = 'three'
    c.add('three', ret)
r1 = c('four', lambda x:f'value: {x}', 4)
r2 = c('four', lambda x:f'value: {x}', 5) #should not get called
assert r1==r2
print(c)
c.remove('four')
assert 'four' not in c
print(c)

Clearing cache!
{'one': Path('/tmp/test_cache/cache_file_x7d12ee9d3b7cf548.pkl'), 'two': Path('/tmp/test_cache/cache_file_x701df7dd200ae1f.pkl'), 'three': Path('/tmp/test_cache/cache_file_x1eb9e18e1559b080.pkl'), 'four': Path('/tmp/test_cache/cache_file_x57b2e4450c14754d.pkl')}
{'one': Path('/tmp/test_cache/cache_file_x7d12ee9d3b7cf548.pkl'), 'two': Path('/tmp/test_cache/cache_file_x701df7dd200ae1f.pkl'), 'three': Path('/tmp/test_cache/cache_file_x1eb9e18e1559b080.pkl')}


In [ ]:
#export
@dataclass
class Files:
    """ paths to the various files that we need"""

    data:str = '$HOME/data'
    ft2: str = '$HOME/work/lat-data/ft2'
    gti: str = '$HOME/work/lat-data/binned/'
    aeff:str = '$HOME/work/lat-data/aeff'
    weights: str = '$HOME/onedrive/fermi/weight_files'
    cachepath: str = '/tmp/lc_cache'

    # expand -- not implemented in Path
    def __post_init__(self):
        d = self.__dict__
        for name, value in d.items():
            d[name] = Path(os.path.expandvars(value))
        self.cachepath.mkdir(exist_ok=True)
        self.cache = _Cache(self.cachepath, clear=False)
        
    @property
    def valid(self):
        """assume all files ok if aeff"""
        return self.aeff.exists()
    
    def __repr__(self):
        s = 'File paths for light curves\n'
        for name, value in self.__dict__.items():
            s += f'  {name:10s} : {value}\n'
        return s

In [ ]:
files = Files()
files

File paths for light curves
  data       : /home/burnett/data
  ft2        : /home/burnett/work/lat-data/ft2
  gti        : /home/burnett/work/lat-data/binned
  aeff       : /home/burnett/work/lat-data/aeff
  weights    : /home/burnett/onedrive/fermi/weight_files
  cachepath  : /tmp/lc_cache
  cache      : {}

In [ ]:
#export                
class PointSource():
    """Manage the position and name of a point source
    """
    def __init__(self, name, position=None):
        """position: (l,b) tuple or None. if None, expect to be found by lookup
        """
        self.name=name
        if position is None:
            skycoord = SkyCoord.from_name(name)
            gal = skycoord.galactic
            self.l,self.b = (gal.l.value, gal.b.value)
        else:
            self.l,self.b = position
            skycoord = SkyCoord(self.l,self.b, unit='deg', frame='galactic')
        self.skycoord = skycoord
    def __str__(self):
        return f'Source "{self.name}" at: (l,b)=({self.l:.3f},{self.b:.3f})'
    def __repr__(self): return str(self)

    @property
    def ra(self):
        sk = self.skycoord.transform_to('fk5')
        return sk.ra.value
    @property
    def dec(self):
        sk = self.skycoord.transform_to('fk5')
        return sk.dec.value

    @property
    def filename(self):
        """Modified name for file system"""
        return self.name.replace(' ', '_').replace('+','p')

    @classmethod
    def fk5(cls, name, position):
        """position: (ra,dec) tuple """
        ra,dec = position
        sk = SkyCoord(ra, dec, unit='deg',  frame='fk5').transform_to('galactic')
        return cls(name, (sk.l.value, sk.b.value))

In [ ]:
show_doc(PointSource.fk5)

<h4 id="PointSource.fk5" class="doc_header"><code>PointSource.fk5</code><a href="__main__.py#L35" class="source_link" style="float:right">[source]</a></h4>

> <code>PointSource.fk5</code>(**`name`**, **`position`**)

position: (ra,dec) tuple 

In [ ]:
for s, expect in [( PointSource('Geminga'),             'Source "Geminga" at: (l,b)=(195.134,4.266)'),
                  ( PointSource('gal_source', (0,0)),   'Source "gal_source" at: (l,b)=(0.000,0.000)', ),
                  ( PointSource.fk5('fk5_source',(0,0)),'Source "fk5_source" at: (l,b)=(96.337,-60.189)',)
                   ]:    
    assert str(s)==expect, f'expected {expect}, got {str(s)}'


In [ ]:
files = Files()
files.cache

{}

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_load_gti.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 07_cells.ipynb.
Converted 09_poisson.ipynb.
Converted 10_loglike.ipynb.
Converted 11_lightcurve.ipynb.
Converted 12_instructions.ipynb.
Converted 13_kerr_comparison.ipynb.
Converted 14_bayesian.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.
Mon Dec 14 10:46:22 PST 2020
